In [20]:
# Cell 1: Install and import
import asyncio
import websockets
import json
import ssl
import base64
import nest_asyncio
# Enable nested event loops for notebooks
nest_asyncio.apply()

# Your HF Space URL (update this!)
WS_URL = "wss://ymhhvwsjp7v90d-8080.proxy.runpod.net/ws"
print(f"🎯 Target: {WS_URL}")

🎯 Target: wss://ymhhvwsjp7v90d-8080.proxy.runpod.net/ws


In [16]:
# Enable nested event loops for notebooks

def create_ssl_context():
    """Create SSL context that disables certificate verification for testing"""
    ssl_context = ssl.create_default_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE
    return ssl_context

async def connect_websocket(url, timeout=10.0):
    """
    Connect to WebSocket with SSL certificate handling
    
    Args:
        url (str): WebSocket URL (wss:// or ws://)
        timeout (float): Connection timeout in seconds
    
    Returns:
        websocket: Connected WebSocket object or None if failed
    """
    try:
        print(f"🔗 Connecting to: {url}")
        
        # Create SSL context for secure connections
        ssl_context = None
        if url.startswith('wss://'):
            ssl_context = create_ssl_context()
            print("🔒 Using SSL context with certificate verification disabled")
        
        # Connect with timeout
        websocket = await asyncio.wait_for(
            websockets.connect(
                url,
                ping_interval=20,
                ping_timeout=20,
                ssl=ssl_context
            ),
            timeout=timeout
        )
        
        print("✅ Connected successfully!")
        return websocket
        
    except asyncio.TimeoutError:
        print("❌ Connection timeout")
        return None
    except ssl.SSLError as e:
        print(f"❌ SSL Error: {e}")
        return None
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return None

In [17]:
async def send_message(websocket, message, timeout=10.0):
    """
    Send a message and wait for response
    
    Args:
        websocket: Connected WebSocket object
        message (dict): Message to send
        timeout (float): Response timeout in seconds
    
    Returns:
        dict: Response message or None if failed
    """
    try:
        # Send message
        await websocket.send(json.dumps(message))
        print(f"📤 Sent: {message}")
        
        # Wait for response
        response = await asyncio.wait_for(websocket.recv(), timeout=timeout)
        result = json.loads(response)
        print(f"📥 Received: {result}")
        
        return result
        
    except asyncio.TimeoutError:
        print("❌ Response timeout")
        return None
    except Exception as e:
        print(f"❌ Send/Receive failed: {e}")
        return None
        
async def test_connection(url="wss://ymhhvwsjp7v90d-8080.proxy.runpod.net/ws"):
    """
    Test WebSocket connection with ping/pong
    
    Args:
        url (str): WebSocket URL to test
    
    Returns:
        bool: True if connection successful, False otherwise
    """
    websocket = await connect_websocket(url)
    if websocket is None:
        return False
    
    try:
        # Send ping
        response = await send_message(websocket, {"type": "ping"})
        if response and response.get("type") == "pong":
            print("✅ Ping/Pong successful!")
            return True
        else:
            print("❌ Unexpected response")
            return False
    finally:
        await websocket.close()
        print("🔌 Connection closed")

In [21]:

async def test_connection(url="wss://ymhhvwsjp7v90d-8080.proxy.runpod.net/ws"):
    """
    Test WebSocket connection with ping/pong
    
    Args:
        url (str): WebSocket URL to test
    
    Returns:
        bool: True if connection successful, False otherwise
    """
    websocket = await connect_websocket(url)
    if websocket is None:
        return False
    
    try:
        # Send ping
        response = await send_message(websocket, {"type": "ping"})
        if response and response.get("type") == "pong":
            print("✅ Ping/Pong successful!")
            return True
        else:
            print("❌ Unexpected response")
            return False
    finally:
        await websocket.close()
        print("🔌 Connection closed")

# Convenience function for notebooks
def test_websocket(url="wss://ymhhvwsjp7v90d-8080.proxy.runpod.net/ws"):
    """Synchronous wrapper for testing WebSocket connection"""
    return asyncio.run(test_connection(url)) 

test_websocket()

🔗 Connecting to: wss://ymhhvwsjp7v90d-8080.proxy.runpod.net/ws
🔒 Using SSL context with certificate verification disabled
✅ Connected successfully!
📤 Sent: {'type': 'ping'}
📥 Received: {'type': 'pong'}
✅ Ping/Pong successful!
🔌 Connection closed


True

In [22]:
# Test audio transcription
AUDIO_FILE = "voice/a_projectile_is.wav"  # Update your path

async def test_audio():
    try:
        print(f"🎤 Testing audio: {AUDIO_FILE}")
        
        with open(AUDIO_FILE, "rb") as f:
            audio_data = base64.b64encode(f.read()).decode()
        
        print(f"📊 Audio size: {len(audio_data)} chars")
        
        # Create SSL context for secure connections
        ssl_context = None
        if WS_URL.startswith('wss://'):
            ssl_context = ssl.create_default_context()
            ssl_context.check_hostname = False
            ssl_context.verify_mode = ssl.CERT_NONE
            print("🔒 Using SSL context with certificate verification disabled")
        
        async with websockets.connect(WS_URL, ssl=ssl_context) as websocket:
            print("✅ Connected to WebSocket!")
            print("📤 Sending audio...")
            
            await websocket.send(json.dumps({
                "audio_data": audio_data,
                "mime_type": "audio/wav"
            }))
            
            print("⏳ Waiting for transcription...")
            response = await websocket.recv()
            result = json.loads(response)
            
            print("📥 Transcription:")
            print(f"   Type: {result.get('type')}")
            print(f"   Text: '{result.get('text')}'")
            
    except ssl.SSLError as e:
        print(f"❌ SSL Error: {e}")
    except websockets.exceptions.ConnectionClosed as e:
        print(f"❌ Connection closed: {e}")
    except asyncio.TimeoutError:
        print("❌ Connection timeout")
    except Exception as e:
        print(f"❌ Error: {e}")

asyncio.run(test_audio())

🎤 Testing audio: voice/a_projectile_is.wav
📊 Audio size: 1169176 chars
🔒 Using SSL context with certificate verification disabled
✅ Connected to WebSocket!
📤 Sending audio...
⏳ Waiting for transcription...
📥 Transcription:
   Type: transcription
   Text: 'A projectile is any object that is thrown or projected into the air and is only subject to the force of gravity. Projectile motion is the specific type of motion that a projectile undergoes, following a curved path.'
